<a href="https://colab.research.google.com/github/BogdanGFTP/fastai_course/blob/master/imdb_les4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import tarfile
tar = tarfile.open("gdrive/My Drive/Fast.ai course files/aclImdb.tar")
tar.extractall()
tar.close()

In [3]:
!pip install "fastai==0.7.0"
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
!pip install Pillow>=4.1.1
!pip install image
!pip install torchtext==0.2.3

!pip install seaborn --upgrade matplotlib
!pip install seaborn --upgrade pandas
!pip install seaborn --upgrade pillow
!pip install seaborn --upgrade plotly

    100% |████████████████████████████████| 122kB 3.8MB/s 
    100% |████████████████████████████████| 1.5MB 16.7MB/s 
    100% |████████████████████████████████| 71kB 26.3MB/s 
    100% |████████████████████████████████| 184kB 30.2MB/s 
    100% |████████████████████████████████| 61kB 24.1MB/s 
    100% |████████████████████████████████| 496.4MB 36kB/s 
    100% |████████████████████████████████| 3.6MB 5.5MB/s 
    100% |████████████████████████████████| 92kB 30.1MB/s 
    100% |████████████████████████████████| 11.6MB 2.2MB/s 
    100% |████████████████████████████████| 61kB 17.8MB/s 
    100% |████████████████████████████████| 81kB 25.1MB/s 
  Running setup.py bdist_wheel for bcolz ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
  Stored in directory: /root/.cache/pip/wheels/9f/78/26/fb8c0

In [0]:
# !pip install pillow==4.1.1

In [0]:
# !{sys.executable} -m pip install pyarrow==0.10.0

In [0]:
# !pip install pandas --upgrade

In [0]:
# from PIL import Image
import matplotlib.pyplot as plt

In [5]:
!wget -O /usr/local/lib/python3.6/dist-packages/fastai/weights.tgz http://files.fast.ai/models/weights.tgz

--2019-01-15 14:01:49--  http://files.fast.ai/models/weights.tgz
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1195411318 (1.1G) [text/plain]
Saving to: ‘/usr/local/lib/python3.6/dist-packages/fastai/weights.tgz’

/usr/local/lib/pyth 100%[===================>]   1.11G  15.6MB/s    in 74s     

2019-01-15 14:03:04 (15.3 MB/s) - ‘/usr/local/lib/python3.6/dist-packages/fastai/weights.tgz’ saved [1195411318/1195411318]



In [6]:
!tar xvfz /usr/local/lib/python3.6/dist-packages/fastai/weights.tgz -C /usr/local/lib/python3.6/dist-packages/fastai

weights/
weights/wrn_50_2f.pth
weights/resnext_101_32x4d.pth
weights/resnext_101_64x4d.pth
weights/inceptionresnetv2-d579a627.pth
weights/inceptionv4-97ef9c30.pth
weights/resnext_50_32x4d.pth


In [0]:
from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy

In [0]:
PATH = 'aclImdb/'

TRN_PATH = 'train/all/'
VAL_PATH = 'test/all/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

In [10]:
trn_files = !ls {TRN}
trn_files[:10]

['0_0.txt       1562_10.txt  24997_0.txt\t34371_0.txt  43748_0.txt  6248_7.txt',
 '0_3.txt       15621_0.txt  24998_0.txt\t3437_1.txt   43749_0.txt  6249_0.txt',
 '0_9.txt       1562_1.txt   24999_0.txt\t34372_0.txt  437_4.txt\t  6249_2.txt',
 '10000_0.txt   15622_0.txt  25000_0.txt\t34373_0.txt  43750_0.txt  6249_7.txt',
 '10000_4.txt   15623_0.txt  2500_0.txt\t34374_0.txt  4375_0.txt   624_9.txt',
 '10000_8.txt   15624_0.txt  25001_0.txt\t34375_0.txt  43751_0.txt  6250_0.txt',
 '1000_0.txt    15625_0.txt  2500_1.txt\t34376_0.txt  4375_1.txt   6250_10.txt',
 '10001_0.txt   15626_0.txt  25002_0.txt\t34377_0.txt  43752_0.txt  6250_1.txt',
 '10001_10.txt  15627_0.txt  25003_0.txt\t34378_0.txt  43753_0.txt  625_0.txt',
 '10001_4.txt   15628_0.txt  25004_0.txt\t3437_8.txt   43754_0.txt  625_10.txt']

In [11]:
review = !cat {TRN}{trn_files[6]}
review[0]

"I have to say when a name like Zombiegeddon and an atom bomb on the front cover I was expecting a flat out chop-socky fung-ku, but what I got instead was a comedy. So, it wasn't quite was I was expecting, but I really liked it anyway! The best scene ever was the main cop dude pulling those kids over and pulling a Bad Lieutenant on them!! I was laughing my ass off. I mean, the cops were just so bad! And when I say bad, I mean The Shield Vic Macky bad. But unlike that show I was laughing when they shot people and smoked dope.<br /><br />Felissa Rose...man, oh man. What can you say about that hottie. She was great and put those other actresses to shame. She should work more often!!!!! I also really liked the fight scene outside of the building. That was done really well. Lots of fighting and people getting their heads banged up. FUN! Last, but not least Joe Estevez and William Smith were great as the...well, I wasn't sure what they were, but they seemed to be having fun and throwing out 

In [12]:
# How many words are in the ds
!find {TRN} -name '*.txt' | xargs cat | wc -w

17486581


In [13]:
!find {VAL} -name '*.txt' | xargs cat | wc -w

5686719


Tokenizing text

In [0]:
# $ python -m spacy download en

In [0]:
spacy_tok = spacy.load('en')

In [15]:
' '.join([sent.string.strip() for sent in spacy_tok(review[0])])

"I have to say when a name like Zombiegeddon and an atom bomb on the front cover I was expecting a flat out chop - socky fung - ku , but what I got instead was a comedy . So , it was n't quite was I was expecting , but I really liked it anyway ! The best scene ever was the main cop dude pulling those kids over and pulling a Bad Lieutenant on them ! ! I was laughing my ass off . I mean , the cops were just so bad ! And when I say bad , I mean The Shield Vic Macky bad . But unlike that show I was laughing when they shot people and smoked dope.<br /><br />Felissa Rose ... man , oh man . What can you say about that hottie . She was great and put those other actresses to shame . She should work more often ! ! ! ! ! I also really liked the fight scene outside of the building . That was done really well . Lots of fighting and people getting their heads banged up . FUN ! Last , but not least Joe Estevez and William Smith were great as the ... well , I was n't sure what they were , but they see

In [0]:
TEXT = data.Field(lower=True, tokenize='spacy')

In [0]:
bs = 64
bptt = 70

In [0]:
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

In [0]:
!mkdir aclImdb/models

In [0]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl', 'wb'))

In [24]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(4583, 37392, 1, 20540756)

In [25]:
# 'itos': 'int-to-string'
TEXT.vocab.itos[:12]

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is', 'in', 'it']

In [27]:
# 'stoi': 'string-to-int'
TEXT.vocab.stoi['the']

2

In [28]:
md.trn_ds[0].text[:12]

['this',
 'movie',
 'made',
 'it',
 'into',
 'one',
 'of',
 'my',
 'top',
 '10',
 'most',
 'awful']

In [29]:
TEXT.numericalize([md.trn_ds[0].text[:12]])

Variable containing:
   13
   23
  107
   11
   95
   37
    7
   76
  367
  328
  104
  402
[torch.cuda.LongTensor of size 12x1 (GPU 0)]

In [30]:
next(iter(md.trn_dl))

(Variable containing:
     13   1831      3  ...       4    451     14
     23   1081    316  ...      83   6063      9
    107     13   3836  ...     161  11295   4055
         ...            ⋱           ...         
     93   1869   3836  ...       6   1288     20
   1682   1991    177  ...     356     46      6
     14   1490     27  ...       7    299   3470
 [torch.cuda.LongTensor of size 71x64 (GPU 0)], Variable containing:
     23
   1081
    316
   ⋮   
   2128
    123
  24435
 [torch.cuda.LongTensor of size 4544 (GPU 0)])

Train

In [0]:
em_sz = 200 # size of each embedding vector
nh = 500    # number of hidden activations per layer
nl = 3      # number of layers

In [0]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [0]:
learner = md.get_model(opt_fn, em_sz, nh, nl, dropouti=0.05,
                      dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [34]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   
    0      4.857316   4.760821  
    1      4.647547   4.536541  
    2      4.526532   4.451085  
    3      4.583891   4.470384  
    4      4.495273   4.397718  
    5      4.420798   4.342571  
    6      4.393184   4.325964  
    7      4.515738   4.414149  
    8      4.48249    4.387429  
    9      4.44446    4.359053  
    10     4.413453   4.325632  
    11     4.365878   4.297578  
    12     4.3346     4.273429  
    13     4.326683   4.260329  
    14     4.283883   4.256827  



[array([4.25683])]

In [0]:
learner.save_encoder('adam1_enc')

In [0]:
learner.load_encoder('adam1_enc')

In [37]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=10)

epoch      trn_loss   val_loss   
    0      4.459133   4.368936  
    1      4.449753   4.358021  
    2      4.425812   4.341074  
    3      4.395653   4.32034   
    4      4.365483   4.295221  
    5      4.334033   4.272818  
    6      4.306159   4.252659  
    7      4.285843   4.238028  
    8      4.296537   4.231432  
    9      4.28098    4.229016  



[array([4.22902])]

In [0]:
learner.save_encoder('adam3_10_enc')

In [0]:
learner.load_encoder('adam3_10_enc')

In [40]:
math.exp(4.165)

64.3926824434624

In [0]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl', 'wb'))

Test

In [42]:
m = learner.model
ss = """. So, it wasn't quite was I expecting, but I really liked it anyway! The best"""
s = [TEXT.preprocess(ss)]
t = TEXT.numericalize(s)
' '.join(s[0])

". so , it was n't quite was i expecting , but i really liked it anyway ! the best"

In [0]:
# Set batch to size 1
m[0].bs = 1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res, *_ = m(t)
# Put the batch size back to what it was
m[0].bs = bs

In [44]:
# Top 10 predictions
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['part',
 'thing',
 'way',
 'scene',
 'parts',
 'of',
 'scenes',
 'movie',
 'aspect',
 'i']

In [45]:
print(ss, '\n')
for i in range(50):
  n = res[-1].topk(2)[1]
  n = n[1] if n.data[0]==0 else n[0]
  print(TEXT.vocab.itos[n.data[0]], end=' ')
  res, *_ = m(n[0].unsqueeze(0))
print('...')

. So, it wasn't quite was I expecting, but I really liked it anyway! The best 

part of the story , and the story is about a young girl who is a woman who is a woman who is a woman . she is a woman who is a woman and is a woman who is a woman . she is a woman who is a ...


In [0]:
TEXT = pickle.load(open(f'{PATH}models/TEXT.pkl', 'rb'))

In [47]:
IMDB_LABEL = data.Field(sequential=False)
splits = torchtext.datasets.IMDB.splits(TEXT, IMDB_LABEL, 'data/')

downloading aclImdb_v1.tar.gz


In [0]:
t = splits[0].examples[0]

In [49]:
t.label, ' '.join(t.text[:16])

('pos',
 "i think ' blackadder the third ' is the best one of the series.<br /><br />actuelly")

In [0]:
md2 = TextData.from_splits(PATH, splits, bs)

In [0]:
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh,
                  n_layers=nl, dropout=0.1, dropouti=0.4,
                  wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder(f'adam3_10_enc')

In [0]:
m3.clip = 25.
lrs = np.array([1e-4, 1e-4, 1e-4, 1e-3, 1e-2])

In [53]:
m3.freeze_to(-1)
m3.fit(lrs/2, 1, metrics=[accuracy])
m3.unfreeze()
m3.fit(lrs, 1, metrics=[accuracy], cycle_len=1)

epoch      trn_loss   val_loss   accuracy   
    0      0.673538   0.4045     0.819744  



epoch      trn_loss   val_loss   accuracy   
    0      0.48765    0.299093   0.880618  



[array([0.29909]), 0.8806182960591874]

In [0]:
m3.fit(lrs, 7, metrics=[accuracy], cycle_len=1, cycle_save_name='imdb2')

epoch      trn_loss   val_loss   accuracy   
    0      0.427399   0.296208   0.889011  
    1      0.394796   0.314634   0.881915  
    2      0.372595   0.278263   0.89842   
    3      0.354477   0.298867   0.892296  
 78%|███████▊  | 305/391 [08:46<02:23,  1.67s/it, loss=0.355]

In [0]:
m3.load_cycle('imdb2', 4)

In [0]:
accuracy_np(*m3.predict_with_targs())